In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import probnum as pn

import linpde_gp

In [ ]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0000_poisson_dirichlet_1d"
config.target = "jmlr"
config.debug_mode = True

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams.update(config.tueplots_bundle())

## Problem Definition

In [ ]:
domain = linpde_gp.domains.asdomain([-1.0, 1.0])

In [ ]:
# Boundary Values
g = np.asarray((0.0, 1.0))

# PDE RHS
f = linpde_gp.functions.Constant(input_shape=(), value=2.0)

# True Solution
u = linpde_gp.problems.pde.PoissonEquation1DConstRHSDirichletProblemSolution(
    domain=domain,
    rhs=f.value,
    boundary_values=g,
)

# PDE Measurements
X_pde = np.linspace(-0.8, 0.8, 3)

In [ ]:
# Boundary Values
g = np.zeros(2)

# PDE RHS
f = pn.LambdaFunction(lambda x: np.pi ** 2 * np.sin(np.pi * x), input_shape=())

# True Solution
u = pn.LambdaFunction(lambda x: np.sin(np.pi * x), input_shape=())

# PDE Measurements
X_pde = np.linspace(-0.8, 0.8, 3)

In [ ]:
bvp = linpde_gp.problems.pde.PoissonEquationDirichletProblem(
    domain=domain,
    rhs=linpde_gp.randprocs.DeterministicProcess(f),
    boundary_values=pn.randvars.asrandvar(g),
    solution=u,
)

f_X_pde = bvp.rhs(X_pde)

In [ ]:
class BeliefPlotter:
    def __init__(self, bvp: linpde_gp.problems.pde.PoissonEquationDirichletProblem):
        self._bvp = bvp
        self._plt_grid = np.linspace(*bvp.domain, 100)

    def plot_belief(
        self,
        ax,
        u: pn.randprocs.GaussianProcess,
        conditioned_on: list[str] = [],
        # X_pde: np.ndarray | None = None,
        # f_X_pde: pn.randvars.RandomVariable | None = None,
        X_pde: np.ndarray = None,
        f_X_pde: pn.randvars.RandomVariable = None,
    ):
        u_conditional_strs = []
        
        for key in conditioned_on:
            if key == "bc":
                u_conditional_strs.append(r"u\vert_{\partial \Omega} - g = 0")
            elif key == "pde":
                u_conditional_strs.append(r"-\Delta u(x_i) - f(x_i) = 0")
        
        u_label = (
            fr"$u \mid {', '.join(u_conditional_strs)}$"
            if len(u_conditional_strs) > 0
            else "$u$"
        )
        
        u.plot(
            ax,
            self._plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C0",
            label=u_label,
        )
        
        ax.plot(
            self._plt_grid,
            bvp.solution(self._plt_grid),
            color="C1",
            label="$u^*$",
        )
        
        for key in conditioned_on:
            if key == "bc":
                g = bvp.boundary_conditions[0].values

                ax.errorbar(
                    bvp.domain.boundary,
                    g.mean,
                    yerr=1.96 * g.std,
                    fmt="+",
                    capsize=2,
                    color="C2",
                    label=r"$g$",
                )
            elif key == "pde":
                linpde_gp.utils.plotting.plot_local_curvature(
                    ax,
                    xs=X_pde,
                    f_xs=u.mean(X_pde),
                    ddf_xs=-f_X_pde,
                    df_xs=(
                        linpde_gp.linfuncops.diffops.PartialDerivative(
                            domain_shape=(),
                            domain_index=(),
                        )(u)(X_pde).mean
                    ),
                    color="C3",
                    label=f"$(f(x_1), \dots, f(x_{X_pde.shape[0]}))$",
                )
        
        ax.legend()

    def plot_pred_belief(
        self,
        ax,
        u: pn.randprocs.GaussianProcess,
        conditioned_on: list[str] = [],
        # X_pde: np.ndarray | None = None,
        # f_X_pde: pn.randvars.RandomVariable | None = None,
        X_pde: np.ndarray = None,
        f_X_pde: pn.randvars.RandomVariable = None,
    ):
        u_conditional_strs = []
        
        for key in conditioned_on:
            if key == "bc":
                u_conditional_strs.append(r"u\vert_{\partial \Omega} - g = 0")
            elif key == "pde":
                u_conditional_strs.append(r"-\Delta u(x_i) - f(x_i) = 0")
        
        u_label = (
            fr"$-\Delta u \mid {', '.join(u_conditional_strs)}$"
            if len(u_conditional_strs) > 0
            else "$-\Delta u$"
        )

        bvp.diffop(u).plot(
            ax,
            self._plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C0",
            label=u_label,
        )
        
        self._bvp.rhs.plot(
            ax,
            self._plt_grid,
            color="C1",
            label="$f$",
        )
        
        if "pde" in conditioned_on:
            ax.errorbar(
                X_pde,
                f_X_pde.mean,
                yerr=1.96 * f_X_pde.std,
                fmt="+",
                capsize=2,
                c="C3",
                label=f"$(f(x_1), \dots, f(x_{X_pde.shape[0]}))$",
            )
        
        ax.legend()

plotter = BeliefPlotter(bvp)

## Prior

In [ ]:
u_prior = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Zero(input_shape=()),
    cov=2.0 ** 2 * linpde_gp.randprocs.kernels.ExpQuad(
        input_shape=(),
        lengthscales=1.0,
    ),
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_prior,
)

experiment_utils.savefig("00a_u_prior")

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_prior,
)

experiment_utils.savefig("00b_Du_prior")

## Posterior (PDE First)

### Conditioning on the PDE

In [ ]:
u_cond_pde = u_prior.condition_on_observations(
    X=X_pde,
    Y=np.zeros_like(X_pde),
    L=bvp.diffop,
    b=-f_X_pde,
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_pde,
    conditioned_on=["pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

experiment_utils.savefig("pdefirst_01a_u_cond_pde")

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde,
    conditioned_on=["pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

experiment_utils.savefig("pdefirst_01b_Du_cond_pde")

### Conditioning on the Boundary Conditions

In [ ]:
u_cond_pde_bc = u_cond_pde.condition_on_observations(
    X=bvp.domain.boundary,
    Y=np.zeros_like(bvp.domain.boundary),
    b=-bvp.boundary_conditions[0].values,
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_pde_bc,
    conditioned_on=["pde", "bc"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

experiment_utils.savefig("pdefirst_02a_u_cond_pde_bc")

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde_bc,
    conditioned_on=["pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

experiment_utils.savefig("pdefirst_02b_Du_cond_pde_bc")

### Complete Plot

In [ ]:
for include_bc in [False, True]:
    nrows = 3 if include_bc else 2
    
    rc = config.tueplots_bundle(nrows=nrows, ncols=2)
    rc.update(
        {
            "lines.linewidth": 1
        }
    )
    
    with plt.rc_context(rc):
        fig, ax = plt.subplots(nrows=nrows, ncols=2)

        ax[0, 0].set_title("(a)")

        plotter.plot_belief(
            ax=ax[0, 0],
            u=u_prior,
        )

        ax[0, 1].set_title("(b)")

        plotter.plot_pred_belief(
            ax=ax[0, 1],
            u=u_prior,
        )

        ax[1, 0].set_title("(c)")

        plotter.plot_belief(
            ax=ax[1, 0],
            u=u_cond_pde,
            conditioned_on=["pde"],
            X_pde=X_pde,
            f_X_pde=f_X_pde,
        )

        ax[1, 1].set_title("(d)")

        plotter.plot_pred_belief(
            ax=ax[1, 1],
            u=u_cond_pde,
            conditioned_on=["pde"],
            X_pde=X_pde,
            f_X_pde=f_X_pde,
        )
        
        if include_bc:
            ax[2, 0].set_title("(e)")

            plotter.plot_belief(
                ax=ax[2, 0],
                u=u_cond_pde_bc,
                conditioned_on=["pde", "bc"],
                X_pde=X_pde,
                f_X_pde=f_X_pde,
            )

            ax[2, 1].set_title("(f)")

            plotter.plot_pred_belief(
                ax=ax[2, 1],
                u=u_cond_pde_bc,
                conditioned_on=["pde", "bc"],
                X_pde=X_pde,
                f_X_pde=f_X_pde,
            )

    experiment_utils.savefig("pdefirst" + ("" if include_bc else "_nobc"))

## Posterior (Boundary Values First)

### Conditioning on Boundary Conditions

In [ ]:
u_cond_bc = u_prior.condition_on_observations(
    bvp.domain.boundary,
    Y=np.zeros_like(bvp.domain.boundary),
    b=-bvp.boundary_conditions[0].values,
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_bc,
    conditioned_on=["bc"],
)

experiment_utils.savefig("bcfirst_01a_u_cond_bc")

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_bc,
    conditioned_on=["bc"],
)

experiment_utils.savefig("bcfirst_01b_Du_cond_bc")

### Conditioning on the PDE

In [ ]:
u_cond_bc_pde = u_cond_bc.condition_on_observations(
    X_pde,
    Y=np.zeros_like(X_pde),
    L=bvp.diffop,
    b=-f_X_pde,
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_bc_pde,
    conditioned_on=["bc", "pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

experiment_utils.savefig("bcfirst_02a_u_cond_bc_pde")

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_bc_pde,
    conditioned_on=["bc", "pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

experiment_utils.savefig("bcfirst_02b_Du_cond_bc_pde")

### Complete Plot

In [ ]:
rc = config.tueplots_bundle(nrows=3, ncols=2)
rc.update(
    {
        "lines.linewidth": 1
    }
)

with plt.rc_context(rc):
    fig, ax = plt.subplots(nrows=3, ncols=2)
    
    ax[0, 0].set_title("(a)")
    
    plotter.plot_belief(
        ax=ax[0, 0],
        u=u_prior,
    )
    
    ax[0, 1].set_title("(b)")
    
    plotter.plot_pred_belief(
        ax=ax[0, 1],
        u=u_prior,
    )
    
    ax[1, 0].set_title("(c)")
    
    plotter.plot_belief(
        ax=ax[1, 0],
        u=u_cond_bc,
        conditioned_on=["bc"],
    )
    
    ax[1, 1].set_title("(d)")
    
    plotter.plot_pred_belief(
        ax=ax[1, 1],
        u=u_cond_bc,
        conditioned_on=["bc"],
    )
    
    ax[2, 0].set_title("(e)")
    
    plotter.plot_belief(
        ax=ax[2, 0],
        u=u_cond_bc_pde,
        conditioned_on=["bc", "pde"],
        X_pde=X_pde,
        f_X_pde=f_X_pde,
    )
    
    ax[2, 1].set_title("(f)")
    
    plotter.plot_pred_belief(
        ax=ax[2, 1],
        u=u_cond_bc_pde,
        conditioned_on=["bc", "pde"],
        X_pde=X_pde,
        f_X_pde=f_X_pde,
    )

experiment_utils.savefig("bcfirst")